# Microsoft Semantic Kernel With IBM Granite Chatbot Notebook

## What is Microsoft Semantic Kernel?

**Microsoft Semantic Kernel (SK)** is an open-source SDK that allows developers to integrate AI capabilities into their applications through a unified framework. It acts as a middleware layer that orchestrates AI models, plugins, and traditional programming logic.

## Key Features & Differentiators

### 🔌 **Plugin Architecture**
- **Semantic Kernel**: Uses a sophisticated plugin system where AI functions can be chained together with traditional code
- **Other Frameworks**: Often focus on single model interactions or require custom integration work

### 🧠 **AI Orchestration** 
- **Semantic Kernel**: Built-in planning capabilities that can automatically sequence multiple AI operations
- **LangChain/LlamaIndex**: Primarily focused on chaining operations manually
- **OpenAI API**: Direct model access without orchestration features

### 🔄 **Multi-Model Support**
- **Semantic Kernel**: Vendor-agnostic - works with OpenAI, Azure OpenAI, Hugging Face, Ollama, and more
- **Other Frameworks**: Often tied to specific providers or require extensive configuration

### 🎯 **Enterprise-Ready**
- **Semantic Kernel**: Built with enterprise needs in mind - security, scalability, and integration with Microsoft ecosystem
- **Other Frameworks**: May require additional tooling for enterprise deployment

### 📝 **Prompt Templates**
- **Semantic Kernel**: Advanced templating system with variable substitution and function calling
- **Other Frameworks**: Basic string formatting or limited templating options

## This Demo

This notebook demonstrates a simple chatbot using Semantic Kernel with:
- **Ollama** as the local AI provider
- **Granite 3.3 8B** model for responses  
- **Prompt templating** for consistent AI behavior
- **Async execution** for responsive interactions

The setup shows how SK abstracts away the complexity of AI integration while providing powerful orchestration capabilities.

In [90]:
# 1. Setup Kernel and Granite 3.3 8B Model for Chatbot
import asyncio
from semantic_kernel import Kernel
from semantic_kernel.prompt_template import PromptTemplateConfig
from semantic_kernel.connectors.ai.ollama import OllamaChatCompletion
from semantic_kernel.connectors.ai.ollama.ollama_prompt_execution_settings import OllamaChatPromptExecutionSettings

kernel = Kernel()

service_id = "ollama"
kernel.add_service(
    OllamaChatCompletion(
        service_id=service_id,
        host="http://localhost:11434",
        ai_model_id="granite3.3:8b",  # Updated to Granite 3.3 8B model [[3]]
    )
)

execution_settings = OllamaChatPromptExecutionSettings()

In [91]:
# 2. Define Dynamic Prompt Template for Chat
prompt_template = """
You are a helpful AI assistant. Please respond to the user's message in a helpful and conversational manner.

User: {{$input}}
"""

prompt_template_config = PromptTemplateConfig(
    template=prompt_template, 
    name="chat", 
    template_format="semantic-kernel"
)

function = kernel.add_function(
    function_name="chat_function",
    plugin_name="chat_plugin",
    prompt_template_config=prompt_template_config,
    prompt_execution_settings=execution_settings,
)

In [94]:
# 3. User Input Cell
user_input = input("Enter your message: ")

In [ ]:
# 4. Execute Chat with Async Support
import textwrap

async def main():
    result = await kernel.invoke(function, input=user_input)
    
    # Format the output with line wrapping
    response_text = str(result)
    wrapped_text = textwrap.fill(response_text, width=80, break_long_words=False, break_on_hyphens=False)
    
    print("Assistant:")
    print(wrapped_text)

asyncio.run(main())